# Decision Trees in Practice

In this assignment we will explore various techniques for preventing overfitting in decision trees. We will extend the implementation of the binary decision trees that we implemented in the previous assignment. <br/>
We will have use our solutions from the previous assignment and extend them.

In this assignment we will:
  -  Implement binary decision trees with different early stopping methods.
  -  Compare models with different stopping parameters.
  -  Visualize the concept of overfitting in decision trees.

Let's get started!

# Fire up Turi Create

Make sure you have the latest version of Turi Create.

In [1]:
import turicreate

In [2]:
import sys
sys.path.append("../common")

from dt_utils import reassign_labels, sub_samples, hot_encode, \
  inter_node_num_mistakes, best_splitting_feature, create_leaf, count_nodes, \
  count_leaves, classify

# Load LendingClub Dataset

This assignment will use the [LendingClub](https://www.lendingclub.com/) dataset used in the previous two assignments.

In [3]:
loans = turicreate.SFrame('../data/lending-club-data.sframe/')

As before, we reassign the labels to have +1 for a safe loan, and -1 for a risky (bad) loan.

In [4]:
loans = reassign_labels(loans)

We will be using the same 4 categorical features as in the previous assignment: 
1. grade of the loan 
2. the length of the loan term
3. the home ownership status: own, mortgage, rent
4. number of years of employment.

In the dataset, each of these features is a categorical feature. Since we are building a binary decision tree, we will have to convert this to binary data in a subsequent section using 1-hot encoding.

In [5]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]

## Subsample dataset to make sure classes are balanced

Just as we did in the previous assignment, we will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. We used `seed = 1` so everyone gets the same results.

In [6]:
loans_data, safe_loans, risky_loans = sub_samples(loans, target, seed=1)

n = len(loans_data)
print(f"Percentage of safe loans                 : {len(safe_loans) / n}")
print(f"Percentage of risky loans                : {len(risky_loans) / n}")
print(f"Total number of loans in our new dataset : {n}")

Percentage of safe loans                 : 0.5022361744216048
Percentage of risky loans                : 0.4977638255783951
Total number of loans in our new dataset : 46508


**Note:** There are many approaches for dealing with imbalanced data, including some where we modify the learning algorithm. These approaches are beyond the scope of this course, but some of them are reviewed in this [paper](http://ieeexplore.ieee.org/xpl/login.jsp?tp=&arnumber=5128907&url=http%3A%2F%2Fieeexplore.ieee.org%2Fiel5%2F69%2F5173046%2F05128907.pdf%3Farnumber%3D5128907 ). For this assignment, we use the simplest possible approach, where we subsample the overly represented class to get a more balanced dataset. In general, and especially when the data is highly imbalanced, we recommend using more advanced methods.

## Transform categorical data into binary features

Since we are implementing **binary decision trees**, we transform our categorical data into binary data using 1-hot encoding, just as in the previous assignment. Here is the summary of that discussion:

For instance, the **home_ownership** feature represents the home ownership status of the loanee, which is either `own`, `mortgage` or `rent`. For example, if a data point has the feature 
```
   {'home_ownership': 'RENT'}
```
we want to turn this into three features: 
```
 { 
   'home_ownership = OWN'      : 0, 
   'home_ownership = MORTGAGE' : 0, 
   'home_ownership = RENT'     : 1
 }
```

Since this code requires a few Python and Turi Create tricks, feel free to use this block of code as is. Refer to the API documentation for a deeper understanding.


In [7]:
loans_data = hot_encode(loans_data, features)
loans_data[0:5]

Columns:
	safe_loans	int
	grade.A	int
	grade.B	int
	grade.C	int
	grade.D	int
	grade.E	int
	grade.F	int
	grade.G	int
	term. 36 months	int
	term. 60 months	int
	home_ownership.MORTGAGE	int
	home_ownership.OTHER	int
	home_ownership.OWN	int
	home_ownership.RENT	int
	emp_length.1 year	int
	emp_length.10+ years	int
	emp_length.2 years	int
	emp_length.3 years	int
	emp_length.4 years	int
	emp_length.5 years	int
	emp_length.6 years	int
	emp_length.7 years	int
	emp_length.8 years	int
	emp_length.9 years	int
	emp_length.< 1 year	int
	emp_length.n/a	int

Rows: 5

Data:
+------------+---------+---------+---------+---------+---------+---------+---------+
| safe_loans | grade.A | grade.B | grade.C | grade.D | grade.E | grade.F | grade.G |
+------------+---------+---------+---------+---------+---------+---------+---------+
|     -1     |    0    |    0    |    1    |    0    |    0    |    0    |    0    |
|     -1     |    0    |    0    |    0    |    0    |    0    |    1    |    0    |
|     -1     |    0    |    1    |    0    |    0    |    0    |    0    |    0    |
|     -1     |    0    |    0    |    1    |    0    |    0    |    0    |    0    |
|     -1     |    0    |    1    |    0    |    0    |    0    |    0    |    0    |
+------------+---------+---------+---------+---------+---------+---------+---------+
+-----------------+-----------------+-------------------------+----------------------+
| term. 36 months | term. 60 months | home_ownership.MORTGAGE | home_ownership.OTHER |
+-----------------+-----------------+-------------------------+----------------------+
|        0        |        1        |            0            |          0           |
|        0        |        1        |            0            |          0           |
|        0        |        1        |            0            |          0           |
|        1        |        0        |            0            |          0           |
|        1        |        0        |            0            |          0           |
+-----------------+-----------------+-------------------------+----------------------+
+--------------------+---------------------+-------------------+----------------------+
| home_ownership.OWN | home_ownership.RENT | emp_length.1 year | emp_length.10+ years |
+--------------------+---------------------+-------------------+----------------------+
|         0          |          1          |         0         |          0           |
|         1          |          0          |         0         |          0           |
|         0          |          1          |         0         |          0           |
|         0          |          1          |         0         |          0           |
|         0          |          1          |         0         |          0           |
+--------------------+---------------------+-------------------+----------------------+
+--------------------+--------------------+--------------------+--------------------+-----+
| emp_length.2 years | emp_length.3 years | emp_length.4 years | emp_length.5 years | ... |
+--------------------+--------------------+--------------------+--------------------+-----+
|         0          |         0          |         0          |         0          | ... |
|         0          |         0          |         1          |         0          | ... |
|         0          |         0          |         0          |         0          | ... |
|         0          |         0          |         0          |         0          | ... |
|         0          |         1          |         0          |         0          | ... |
+--------------------+--------------------+--------------------+--------------------+-----+
[5 rows x 26 columns]

In [8]:
features = loans_data.column_names()
features.remove('safe_loans')  # Remove the response variable
features

['grade.A',
 'grade.B',
 'grade.C',
 'grade.D',
 'grade.E',
 'grade.F',
 'grade.G',
 'term. 36 months',
 'term. 60 months',
 'home_ownership.MORTGAGE',
 'home_ownership.OTHER',
 'home_ownership.OWN',
 'home_ownership.RENT',
 'emp_length.1 year',
 'emp_length.10+ years',
 'emp_length.2 years',
 'emp_length.3 years',
 'emp_length.4 years',
 'emp_length.5 years',
 'emp_length.6 years',
 'emp_length.7 years',
 'emp_length.8 years',
 'emp_length.9 years',
 'emp_length.< 1 year',
 'emp_length.n/a']

## Train-Validation split

We split the data into a train-validation split with 80% of the data in the training set and 20% of the data in the validation set. We use `seed=1` so that everyone gets the same result.

In [9]:
train_data, validation_data = loans_data.random_split(.8, seed=1)

# Early stopping methods for decision trees

In this section, we will extend the **binary tree implementation** from the previous assignment in order to handle some early stopping conditions. Recall the 3 early stopping methods that were discussed in lecture:

1. Reached a **maximum depth**. (set by parameter `max_depth`).
2. Reached a **minimum node size**. (set by parameter `min_node_size`).
3. Don't split if the **gain in error reduction** is too small. (set by parameter `min_error_reduction`).

For the rest of this assignment, we will refer to these three as **early stopping conditions 1, 2, and 3**.

## Early stopping condition 1: Maximum depth

Recall that we already implemented the maximum depth stopping condition in the previous assignment. In this assignment, we will experiment with this condition a bit more and also write code to implement the 2nd and 3rd early stopping conditions.

We will be reusing code from the previous assignment and then building upon this.  We will **alert you** when you reach a function that was part of the previous assignment so that you can simply copy and past your previous code.

## Early stopping condition 2: Minimum node size

The function `reached_minimum_node_size` takes 2 arguments:

1. The `data` (from a node)
2. The minimum number of data points that a node is allowed to split on, `min_node_size`.

This function simply calculates whether the number of data points at a given node is less than or equal to the specified minimum node size. This function will be used to detect this early stopping condition in the `decision_tree_create` function.

In [10]:
def reached_minimum_node_size(data, min_node_size):
    """True if the number of data points is less than or equal to the minimum node size."""
    return len(data) <= min_node_size

**Quiz Question:** Given an intermediate node with 6 safe loans and 3 risky loans, if the `min_node_size` parameter is 10, what should the tree learning algorithm do next?
  - it should stop (early), creating a leaf and return it

## Early stopping condition 3: Minimum gain in error reduction

The function `error_reduction` takes 2 arguments:

1. The error **before** a split, `error_before_split`.
2. The error **after** a split, `error_after_split`.

This function computes the gain in error reduction, i.e., the difference between the error before the split and that after the split. This function will be used to detect this early stopping condition in the `decision_tree_create` function.

In [11]:
def error_reduction(error_before_split, error_after_split):
    """Return the error before the split minus the error after the split."""
    return error_before_split - error_after_split

**Quiz Question:** Assume an intermediate node has 6 safe loans and 3 risky loans.  For each of 4 possible features to split on, the error reduction is 0.0, 0.05, 0.1, and 0.14, respectively. If the **minimum gain in error reduction** parameter is set to 0.2, what should the tree learning algorithm do next?
  - it should stop (early), creating a leaf and return it

## Grabbing binary decision tree helper functions from past assignment

Recall from the previous assignment that we wrote a function `inter_node_num_mistakes` that calculates the number of **misclassified examples** when predicting the **majority class**. This is used to help determine which feature is best to split on at a given node of the tree.

cf. import

In [12]:
## Test case 1
example_labels = turicreate.SArray([-1, -1, 1, 1, 1])
assert inter_node_num_mistakes(example_labels) == 2, 'Test 1 failed... try again!'

## Test case 2
example_labels = turicreate.SArray([-1, -1, 1, 1, -1, 1, 1])
assert inter_node_num_mistakes(example_labels) == 3, 'Test 2 failed... try again!'

## Test case 3
example_labels = turicreate.SArray([-1, -1, -1, -1, 1, -1, 1])
assert inter_node_num_mistakes(example_labels) == 2, 'Test 3 failed... try again!'

We then wrote a function `best_splitting_feature` that finds the best feature to split on given the data and a list of features to consider.

cf. include

In [13]:
act = best_splitting_feature(train_data, features, 'safe_loans') 
exp = 'term. 36 months'
assert act == exp, f"Test failed...Expected {exp}, got: {act}"

Finally, recall the function `create_leaf` from the previous assignment, which creates a leaf node given a set of target values.  

cf. import

## Incorporating new early stopping conditions in binary decision tree implementation

Now, you will implement a function that builds a decision tree handling the three early stopping conditions described in this assignment.  In particular, you will write code to detect early stopping conditions 2 and 3.  You implemented above the functions needed to detect these conditions.  The 1st early stopping condition, **max_depth**, was implemented in the previous assigment and you will not need to reimplement this.  In addition to these early stopping conditions, the typical stopping conditions of having no mistakes or no more features to split on (which we denote by "stopping conditions" 1 and 2) are also included as in the previous assignment.

**Implementing early stopping condition 2: minimum node size:**

* **Step 1:** Use the function **reached_minimum_node_size** that you implemented earlier to write an if condition to detect whether we have hit the base case, i.e., the node does not have enough data points and should be turned into a leaf. Don't forget to use the `min_node_size` argument.
* **Step 2:** Return a leaf. This line of code should be the same as the other (pre-implemented) stopping conditions.


**Implementing early stopping condition 3: minimum error reduction:**

**Note:** This has to come after finding the best splitting feature so we can calculate the error after splitting in order to calculate the error reduction.

* **Step 1:** Calculate the **classification error before splitting**.  Recall that classification error is defined as:

$$
\text{classification error} = \frac{\text{# mistakes}}{\text{# total examples}}
$$
* **Step 2:** Calculate the **classification error after splitting**. This requires calculating the number of mistakes in the left and right splits, and then dividing by the total number of examples.
* **Step 3:** Use the function **error_reduction** to that you implemented earlier to write an if condition to detect whether  the reduction in error is less than the constant provided (`min_error_reduction`). Don't forget to use that argument.
* **Step 4:** Return a leaf. This line of code should be the same as the other (pre-implemented) stopping conditions.


In [14]:
def decision_tree_create(data, features, target, current_depth=0, 
                         max_depth=10, min_node_size=1, 
                         min_error_reduction=0.0, verbose=False):
    
    remaining_features = features[:] # Make a copy of the features.
    target_values = data[target]
    if verbose:
      print("--------------------------------------------------------------------")
      print("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))

    if inter_node_num_mistakes(target_values) == 0:  ## Stopping condition 1: are there mistakes at current node?
        if verbose: print("Stopping condition 1 reached.")
        return create_leaf(target_values)            ## If no mistakes at current node => leaf node
      
    elif len(remaining_features) == 0:               ## Stopping condition 2: are there remaining features to consider splitting on?       
        if verbose: print("Stopping condition 2 reached.")
        return create_leaf(target_values)            ## no remaining features to consider => leaf node
      
    elif current_depth >= max_depth:                 ## Early stopping condition 1: Reached max depth limit.
        if verbose: print("Early stopping condition 1 reached. Reached maximum depth.")
        return create_leaf(target_values)            ## If the max tree depth has been reached =>leaf node    
      
    elif reached_minimum_node_size(data, min_node_size):
        ## Early stopping condition 2: 
        ## If the number of data points is less than or equal to the minimum size, return a leaf.
        if verbose: print("Early stopping condition 2 reached. Reached minimum node size.")
        return create_leaf(target_values) 
    
    ## Find the best splitting feature
    splitting_feature = best_splitting_feature(data, features, target)
    
    ## Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    ## Early stopping condition 3: Minimum error reduction
    ## Calc. the error before splitting = num. of misclassified examples / total number of examples
    err_before_split = inter_node_num_mistakes(target_values) / len(data)
    
    ## Calculate the error after splitting (num. of misclassified examples in both groups / total number of examples)
    left_mistakes = inter_node_num_mistakes(left_split[target]) 
    right_mistakes = inter_node_num_mistakes(right_split[target])
    err_after_split = (left_mistakes + right_mistakes) / len(data)
    
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction, return a leaf.
    if error_reduction(err_before_split, err_after_split) < min_error_reduction:
        if verbose: print("Early stopping condition 3 reached. Minimum error reduction.")
        return create_leaf(target_values)
    
    remaining_features.remove(splitting_feature)
    if verbose:
      print("Split on feature %s. (%s, %s)" % (splitting_feature, len(left_split), len(right_split)))

    ## Repeat (recurse) on left and right subtrees
    l_tree = decision_tree_create(left_split, remaining_features, target, 
                                  current_depth + 1, max_depth, min_node_size, 
                                  min_error_reduction, verbose)        
    
    r_tree = decision_tree_create(right_split, remaining_features, target, 
                                  current_depth + 1, max_depth, min_node_size, 
                                  min_error_reduction, verbose)
    
    return create_leaf(target_values, splitting_feature=splitting_feature, 
                       left=l_tree, right=r_tree, is_leaf=False)

Run the following test code to check your implementation. Make sure you get **'Test passed'** before proceeding.

In [15]:
small_decision_tree = decision_tree_create(train_data, features, 'safe_loans', 
                                           max_depth=2, 
                                           min_node_size=10, min_error_reduction=0.0,
                                           verbose=True)
if count_nodes(small_decision_tree) == 7:
    print('Test passed!')
else:
    print('Test failed... try again!')
    print('Number of nodes found                :', count_nodes(small_decision_tree))
    print('Number of nodes that should be there : 7' )

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 1 (28001 data points).
Split on feature grade.D. (23300, 4701)
--------------------------------------------------------------------
Subtree, depth = 2 (23300 data points).
Early stopping condition 1 reached. Reached maximum depth.
-----------------------------------------------

## Build a tree!

Let's now train a tree model **ignoring early stopping conditions 2 and 3** so that we get the same tree as in the previous assignment.  <br />
To ignore these conditions, we set `min_node_size=0` and `min_error_reduction=-1` (a negative value).

In [16]:
# (no early stopping) 
dt6_model_nes = decision_tree_create(train_data, features, 'safe_loans', max_depth=6, 
                                     min_node_size=0, min_error_reduction=-1.)

Now that your code is working, we will train a tree model on the **train_data** with
* `max_depth = 6`
* `min_node_size = 100`, 
* `min_error_reduction = 0.0`

In [17]:
dt6_model = decision_tree_create(train_data, features, 'safe_loans', max_depth=6, 
                                min_node_size=100, 
                                min_error_reduction=0.0)

## Making predictions

Recall that in the previous assignment you implemented a function `classify` to classify a new point `x` using a given `tree`.

cf. import

Now, let's consider the first example of the validation set and see what the `dt6_model` model predicts for this data point.

In [18]:
validation_data[0]

{'safe_loans': -1,
 'grade.A': 0,
 'grade.B': 0,
 'grade.C': 0,
 'grade.D': 1,
 'grade.E': 0,
 'grade.F': 0,
 'grade.G': 0,
 'term. 36 months': 0,
 'term. 60 months': 1,
 'home_ownership.MORTGAGE': 0,
 'home_ownership.OTHER': 0,
 'home_ownership.OWN': 0,
 'home_ownership.RENT': 1,
 'emp_length.1 year': 0,
 'emp_length.10+ years': 0,
 'emp_length.2 years': 1,
 'emp_length.3 years': 0,
 'emp_length.4 years': 0,
 'emp_length.5 years': 0,
 'emp_length.6 years': 0,
 'emp_length.7 years': 0,
 'emp_length.8 years': 0,
 'emp_length.9 years': 0,
 'emp_length.< 1 year': 0,
 'emp_length.n/a': 0}

In [19]:
print(f"Predicted class: {classify(dt6_model, validation_data[0])}")

Predicted class: -1


Let's add some annotations to our prediction to see what the prediction path was that lead to this predicted class:

In [20]:
classify(dt6_model, validation_data[0], annotate=True)

Split on term. 36 months = 0
Split on grade.A = 0
Split on grade.B = 0
Split on grade.C = 0
Split on grade.D = 1
Split on grade.E = 0
At leaf, predicting -1


-1

Let's now recall the prediction path for the decision tree learned in the previous assignment, which we recreated here as `dt6_model_nes`.

In [21]:
classify(dt6_model_nes, validation_data[0], annotate=True)

Split on term. 36 months = 0
Split on grade.A = 0
Split on grade.B = 0
Split on grade.C = 0
Split on grade.D = 1
Split on grade.E = 0
At leaf, predicting -1


-1

**Quiz Question:** For `dt6_model` trained with `max_depth=6`, `min_node_size=100`, `min_error_reduction=0.0`, is the prediction path for `validation_set[0]` shorter, longer, or the same as for `dt6_model_nes` that ignored the early stopping conditions 2 and 3?
  - it is the same

**Quiz Question:** For `dt6_model` trained with `max_depth=6`, `min_node_size=100`, `min_error_reduction=0.0`, is the prediction path for **any point** always shorter, always longer, always the same, shorter or the same, or longer or the same as for `dt6_model_nes` that ignored the early stopping conditions 2 and 3?
  - It should be shorter or the same

**Quiz Question:** For a tree trained on **any** dataset using `max_depth=6`, `min_node_size=100`, `min_error_reduction=0.0`, what is the maximum number of splits encountered while making a single prediction?
  - max-depth, hence 6 

## Evaluating the model

Now let us evaluate the model that we have trained. You implemented this evaluation in the function `evaluate_classification_error` from the previous assignment.

In [22]:
## to avoid problem with turicreate: ModuleNotFoundError("No module named 'dt_utils'"), when recursing...
def classify_fn(tree, x, annotate=False):
    if tree['is_leaf']:  ## if the node is a leaf node.
      if annotate: print("At leaf, predicting %s" % tree['prediction'])
      return tree['prediction']
    else:
      split_feature_val = x[tree['splitting_feature']]
      if annotate: print("Split on %s = %s" % (tree['splitting_feature'], split_feature_val))
      return classify_fn(tree['left'], x, annotate) if split_feature_val == 0 else \
          classify_fn(tree['right'], x, annotate)

def evaluate_classification_error(tree, data, target):
    preds = data.apply(lambda x: classify_fn(tree, x))    
    ## Then, Calc. the classification error and return it
    num_mistakes = (preds != data[target]).sum()
    return num_mistakes / len(data)

Now, let's use this function to evaluate the classification error of `dt6_model` on the **validation_set**.

In [23]:
round(evaluate_classification_error(dt6_model, validation_data, target), 4)

0.3841

Now, evaluate the validation error using `dt6_model_nes`.

In [24]:
round(evaluate_classification_error(dt6_model_nes, validation_data, target), 4)

0.3838

**Quiz Question:** Is the validation error of the new decision tree (using early stopping conditions 2 and 3) lower than, higher than, or the same as that of the old decision tree from the previous assignment?
  - should be lower!

# Exploring the effect of max_depth

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (**too small**, **just right**, and **too large**).

Train three models with these parameters:

1. **model_1**: max_depth = 2 (too small)
2. **model_2**: max_depth = 6 (just right)
3. **model_3**: max_depth = 14 (may be too large)

For each of these three, we set `min_node_size = 0` and `min_error_reduction = -1`.

**Note:** Each tree can take up to a few minutes to train. In particular, `model_3` will probably take the longest to train.

In [25]:
model_1 = decision_tree_create(train_data, features, 'safe_loans', max_depth=2, 
                               min_node_size=0, min_error_reduction=-1)
model_2 = decision_tree_create(train_data, features, 'safe_loans', max_depth=6, 
                               min_node_size=0, min_error_reduction=-1)
model_3 = decision_tree_create(train_data, features, 'safe_loans', max_depth=14, 
                               min_node_size=0, min_error_reduction=-1)

### Evaluating the models

Let us evaluate the models on the **train** and **validation** data. Let us start by evaluating the classification error on the training data:

In [26]:
def print_eval(models, data, target,
               data_label='training', offset=1, digits=4):
    for (ix, m) in enumerate(models): 
      cr = evaluate_classification_error(m, data, target)
      print(f"{data_label} data, classification error (model {ix + offset}): {round(cr, digits):1.4f}")
    return

In [27]:
 print_eval([model_1, model_2, model_3], train_data, target)

training data, classification error (model 1): 0.4000
training data, classification error (model 2): 0.3819
training data, classification error (model 3): 0.3745


Now evaluate the classification error on the validation data.

In [28]:
print_eval([model_1, model_2, model_3], validation_data, target, data_label='validation')

validation data, classification error (model 1): 0.3981
validation data, classification error (model 2): 0.3838
validation data, classification error (model 3): 0.3800


**Quiz Question:** Which tree has the smallest error on the validation data?
  - model3

**Quiz Question:** Does the tree with the smallest error in the training data also have the smallest error in the validation data?
  - yes!

**Quiz Question:** Is it always true that the tree with the lowest classification error on the **training** set will result in the lowest classification error in the **validation** set?
  - No


### Measuring the complexity of the tree

Recall in the lecture that we talked about deeper trees being more complex. We will measure the complexity of the tree as

```
  complexity(T) = number of leaves in the tree T
```

Compute the number of nodes in `model_1`, `model_2`, and `model_3`.

In [29]:
for (ix, m) in enumerate([model_1, model_2, model_3]):
  cn = count_leaves(m)
  print(f"for model {ix + 1:2d} - num of node is: {cn:4d}")

for model  1 - num of node is:    4
for model  2 - num of node is:   41
for model  3 - num of node is:  341


**Quiz Question:** Which tree has the largest complexity?
  - model 3

**Quiz Question:** Is it always true that the most complex tree will result in the lowest classification error in the **validation_set**?
  - no (most likely it will overfit the training set, and wil generalize poorly).

# Exploring the effect of min_error

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (**negative**, **just right**, and **too positive**).

Train three models with these parameters:
1. **model_4**: `min_error_reduction = -1` (ignoring this early stopping condition)
2. **model_5**: `min_error_reduction = 0` (just right)
3. **model_6**: `min_error_reduction = 5` (too positive)

For each of these three, we set `max_depth = 6`, and `min_node_size = 0`.

In [30]:
model_4 = decision_tree_create(train_data, features, 'safe_loans', max_depth=6, 
                               min_node_size=0, min_error_reduction=-1)
model_5 = decision_tree_create(train_data, features, 'safe_loans', max_depth=6, 
                               min_node_size=0, min_error_reduction=0)
model_6 = decision_tree_create(train_data, features, 'safe_loans', max_depth=6, 
                               min_node_size=0, min_error_reduction=5)

Calculate the accuracy of each model (**model_4**, **model_5**, or **model_6**) on the validation set. 

In [31]:
print_eval([model_4, model_5, model_6], validation_data, target, data_label='validation', offset=4)

validation data, classification error (model 4): 0.3838
validation data, classification error (model 5): 0.3838
validation data, classification error (model 6): 0.5034


Using the `count_leaves` function, compute the number of leaves in each of each models in (**model_4**, **model_5**, and **model_6**). 

In [32]:
for (ix, m) in enumerate([model_4, model_5, model_6]):
  cn = count_leaves(m)
  print(f"for model {ix + 4:2d} - num of node is: {cn:4d}")

for model  4 - num of node is:   41
for model  5 - num of node is:   41
for model  6 - num of node is:    1


**Quiz Question:** Using the complexity definition above, which model (**model_4**, **model_5**, or **model_6**) has the largest complexity?
  - model_4 and model_5

  Did this match your expectation?
  - pretty much
  
**Quiz Question:** **model_4** and **model_5** have similar classification error on the validation set but **model_5** has lower complexity. Should you pick **model_5** over **model_4**?
  - yes

# Exploring the effect of min_node_size

We will compare three models trained with different values of the stopping criterion. Again, intentionally picked models at the extreme ends (**too small**, **just right**, and **just right**).

Train three models with these parameters:
1. **model_7**: min_node_size = 0 (too small)
2. **model_8**: min_node_size = 2000 (just right)
3. **model_9**: min_node_size = 50000 (too large)

For each of these three, we set `max_depth = 6`, and `min_error_reduction = -1`.

In [33]:
model_7 = decision_tree_create(train_data, features, 'safe_loans', max_depth=6, 
                               min_node_size=0, min_error_reduction=-1)
model_8 = decision_tree_create(train_data, features, 'safe_loans', max_depth=6, 
                               min_node_size=2000, min_error_reduction=-1)
model_9 = decision_tree_create(train_data, features, 'safe_loans', max_depth=6, 
                               min_node_size=50000, min_error_reduction=-1)

Now, let us evaluate the models (**model_7**, **model_8**, or **model_9**) on the **validation_set**.

In [34]:
print_eval([model_7, model_8, model_9], validation_data, target, data_label='validation', offset=7)

validation data, classification error (model 7): 0.3838
validation data, classification error (model 8): 0.3845
validation data, classification error (model 9): 0.5034


Using the `count_leaves` function, compute the number of leaves in each of each models (**model_7**, **model_8**, and **model_9**). 

In [35]:
for (ix, m) in enumerate([model_7, model_8, model_9]):
  cn = count_leaves(m)
  print(f"for model {ix + 7:2d} - num of node is: {cn:4d}")

for model  7 - num of node is:   41
for model  8 - num of node is:   19
for model  9 - num of node is:    1


**Quiz Question:** Using the results obtained in this section, which model (**model_7**, **model_8**, or **model_9**) would you choose to use?
  - model_8 (although not clear cut!)